In [1]:
%pip install spacy pymupdf

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import pymupdf  # PyMuPDF
import spacy

# Load SpaCy model
nlp = spacy.load("en_core_web_sm")

def extract_text_from_pdf(pdf_path):
    text = ""
    with pymupdf.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text()
    return text

def extract_entities(text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

def process_documents_in_folder(folder_path):
    processed_docs = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(folder_path, filename)
            text = extract_text_from_pdf(pdf_path)
            entities = extract_entities(text)
            processed_docs.append((pdf_path, entities))
    return processed_docs


In [3]:
%pip install py2neo

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from neo4j import GraphDatabase, basic_auth, Config
import ssl
import logging
uri = "bolt://c24a04a8.databases.neo4j.io:7687"
username = "neo4j"
password = "TIEXesTzQ3V72Od9m-64Kpvd6AxVw8uB4fyvYODD_Jk"

# Create a custom SSL context that skips verification
ssl_context = ssl.create_default_context()
ssl_context.check_hostname = False
ssl_context.verify_mode = ssl.CERT_NONE

# Connect to Neo4j using the custom SSL context
driver = GraphDatabase.driver(uri, auth=basic_auth(username, password), encrypted=True, ssl_context=ssl_context)

def populate_knowledge_graph(docs):
    with driver.session() as session:
        for pdf_path, entities in docs:
            doc_id = pdf_path.split("/")[-1]  # Use file name as document ID
            doc_type = "Invoice" if "invoice" in pdf_path.lower() else "Contract"
            
            for entity, label in entities:
                session.run(
                    "MERGE (n:Entity {name: $name, document_type: $document_type, document_id: $document_id})",
                    name=entity, document_type=doc_type, document_id=doc_id
                )

folder_path = "uploads"
processed_docs = process_documents_in_folder(folder_path)
populate_knowledge_graph(processed_docs)


C:\Users\Abhay\AppData\Local\Temp\ipykernel_22320\3983335161.py:1: DeprecationWarning: Importing Config from neo4j is deprecated without replacement. It's internal and will be removed in a future version.
  from neo4j import GraphDatabase, basic_auth, Config


In [5]:
from fastembed import TextEmbedding

embedding_model = TextEmbedding(model_name="BAAI/bge-small-en-v1.5")

def embed_entities(docs):
    embeddings = {}
    for _, entities in docs:
        for entity, _ in entities:
            if entity not in embeddings:
                embeddings[entity] = embedding_model.embed(entity)
    return embeddings

entity_embeddings = embed_entities(processed_docs)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [11]:
import llm_calls as lc

def retrieve_information(query_text):
    with driver.session() as session:
        result = session.run(
            "MATCH (n) WHERE n.name CONTAINS $query_text RETURN n.name AS name", 
            query_text=query_text
        )
        return [record["name"] for record in result]




query = "Michael Roberts"
retrieved_info = retrieve_information(query)
print(retrieved_info)
print("=="*20)
augmented_input = f"Query: {query}\nKnowledge Graph Info: {retrieved_info}"
response = lc.llm_generate(augmented_input)
print(response)


['Michael Roberts\nAddress', 'Michael Roberts']
It seems like you're looking for information about Michael Roberts! Unfortunately, the given knowledge graph info doesn't provide much detail about who Michael Roberts is or what he's known for.

Can you please provide more context or specify which Michael Roberts you're looking for (e.g., an athlete, actor, historian, etc.)? That would help me provide a more accurate response or assist you in finding the information you need.
